In [2]:
import pandas as pd
import os
import tqdm
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
# Read the data
input_path_base = "/n/holylabs/LABS/cga/Lab/data/geo-tweets/cga-sbg"

output_path_base = "/n/holyscratch01/cga/xiaokang/geo-tweets"

def read_twitter_csv_data(row):
    file_name = row["input_file"]
    
    try:
        df = pd.read_csv(file_name, sep = "\t", lineterminator="\n", dtype="unicode", index_col=None,  compression = "gzip")
    except:
        print("Error reading file: {}".format(file_name))
        return False
    df.drop_duplicates(inplace = True)
    df.to_parquet(row["output_file"], index = False)
    return True


# convert all the files to parquet format

files_df = pd.DataFrame()
for year in range(2010, 2024):
    input_path = os.path.join(input_path_base, str(year))
    output_path = os.path.join(output_path_base, str(year))
    os.makedirs(output_path, exist_ok=True)
    
    
    input_file_list = [os.path.join(input_path, file) for file in os.listdir(input_path) if file.endswith(".csv.gz")]
    file_names = [file.split("/")[-1] for file in input_file_list]
    output_file_names = [os.path.join(output_path, file.split(".csv.gz")[0] + ".parquet") for file in file_names]
    
    files_df = pd.concat([files_df, pd.DataFrame({"input_file": input_file_list, "output_file": output_file_names,"file_name": file_names, "year": year})])


files_df.loc[:, "status"] = files_df.parallel_apply(read_twitter_csv_data, axis = 1)
import datetime
files_df.to_csv(os.path.join(output_path_base, "status_{}.csv".format(datetime.datetime.now().strftime("%Y%m%d%H%M%S"))), index = False)


INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


miss processing data

In [3]:
import pandas as pd
miss_df = pd.read_csv("/n/holyscratch01/cga/xiaokang/geo-tweets/status_20230716174304.csv")
miss_df

,input_file,output_file,file_name,year,status
0,/n/holylabs/LABS/cga/Lab/data/geo-tweets/cga-s...,/n/holyscratch01/cga/xiaokang/geo-tweets/2010/...,2010_9_17_22.csv.gz,2010,True
1,/n/holylabs/LABS/cga/Lab/data/geo-tweets/cga-s...,/n/holyscratch01/cga/xiaokang/geo-tweets/2010/...,2010_12_23_12.csv.gz,2010,True
2,/n/holylabs/LABS/cga/Lab/data/geo-tweets/cga-s...,/n/holyscratch01/cga/xiaokang/geo-tweets/2010/...,2010_11_21_20.csv.gz,2010,True
3,/n/holylabs/LABS/cga/Lab/data/geo-tweets/cga-s...,/n/holyscratch01/cga/xiaokang/geo-tweets/2010/...,2010_7_01_05.csv.gz,2010,True
4,/n/holylabs/LABS/cga/Lab/data/geo-tweets/cga-s...,/n/holyscratch01/cga/xiaokang/geo-tweets/2010/...,2010_4_08_20.csv.gz,2010,True
...,...,...,...,...,...
98533,/n/holylabs/LABS/cga/Lab/data/geo-tweets/cga-s...,/n/holyscratch01/cga/xiaokang/geo-tweets/2023/...,2023_1_28_10.csv.gz,2023,True
98534,/n/holylabs/LABS/cga/Lab/data/geo-tweets/cga-s...,/n/holyscratch01/cga/xiaokang/geo-tweets/2023/...,2023_1_10_12.csv.gz,2023,True
98535,/n/holylabs/LABS/cga/Lab/data/geo-tweets/cga-s...,/n/holyscratch01/cga/xiaokang/geo-tweets/2023/...,2023_4_08_06.csv.gz,2023,True
98536,/n/holylabs/LABS/cga/Lab/data/geo-tweets/cga-s...,/n/holyscratch01/cga/xiaokang/geo-tweets/2023/...,2023_5_23_19.csv.gz,2023,True


In [5]:
miss_df[miss_df["status"] == False]

,input_file,output_file,file_name,year,status
